Install atau import library untuk vader sentiment analysis. Keterangan: google colabs untuk syntax di Linux terminal pake tanda seru

In [1]:
# Install vaderSentiment package for VADER
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.4 MB/s eta 0:00:00


Import library:
pandas : library untuk manipulasi tabel
numpy : library untuk manipulasi array, matrix
sklearn : library untuk machine learning


In [2]:
# Data processing
import pandas as pd
import numpy as np


# Import VADER sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

# Set a wider colwith
pd.set_option('display.max_colwidth', 1000) #pengaturan display

Jika jupiter notebook, file atau data diletakan di folder lokal (komputer). Untuk google colabs, file atau data itu diletakan di google drive

In [3]:
# Mount Google Drive (untuk mengkoneksikan dengan google drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Kita pindah ke lokasi folder dimana terletak file yang ingin kita baca (untuk file google colab nya (.ipynb) letaknya dibawah folder Colab Notebooks)

In [4]:
# Change directory
import os
os.chdir("./drive/My Drive/NLP2023/")

In [5]:
# Print out the current directory
!pwd

/content/drive/My Drive/NLP2023


Sebelum lanjut download datanya dan masukan atau upload ke folder diatas. Datanya ada di github saya: https://github.com/zulkaidaakbar/NLP-STTCipasung2022/tree/main/27-September-2022/VaderSentimentAnalysis

Mengambil data dari google drive kemudian membaca nya (data dalam bentuk text) lalu kita ubah format nya menjadi format tabel menggunakan library pandas

In [6]:
# Read in data
amz_review = pd.read_csv('amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

Kita intip datanya

In [8]:
# Take a look at the data
amz_review.head(10)

,review,label
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up right to get decent volume.,0
6,"If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.",0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


Cari tahu info umum tentang dataset

In [10]:
# Get the dataset information (command .info) -> ternyata ada 1000 entries (1000 reviews)
amz_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [13]:
# cek distribusi sentiment positif dan sentiment negative (dari kolom label)
# Check the label distribution
amz_review['label'].value_counts()

0    500
1    500
Name: label, dtype: int64

Vader adalah library untuk mengetahui sentimen (positif atau negative) dari kalimat atau teks. Vader akan mengeluarkan nilai dari -1 (sangat negative) atau +1 (sentimennya sangat positif). Dibawah ini adalah contoh penggunaan Vader

In [17]:
# Example text
#text = 'GrabNGoInfo.com is a great machine learning tutorial website.'
text = 'The products are expensive and not good'

# VADER Sentiment
vader = SentimentIntensityAnalyzer() #inisilaisasi vader
vader_sentiment = vader.polarity_scores(text) #jalankan vader mengetahui polarity score untuk variabel text


Kita printout hasilnya:
1. Kecenderungan negatif
2. Kecenderungan netral
3. Kecenderungan positif
4. Sentimen keseluruhan (compound score). Ini adalah score total dan biasanya score ini yang digunakan

In [18]:
vader_sentiment # print out hasil score nya

{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.3412}

Karena compound score -0.3412 (negative) oleh karena text diatas sentimen nya negative (jelek)

In [19]:
vader_sentiment['compound'] # untuk print compund score nya saja

-0.3412

Sekarang kita lanjut pakai vader sentiment analysis untuk keseluruhan database

In [20]:
# Get sentiment score for each review
vader_sentiment = SentimentIntensityAnalyzer() #initialisasi
amz_review['scores_VADER'] = amz_review['review'].apply(lambda s: vader_sentiment.polarity_scores(s)['compound']) #mengambil compound score untuk masing2 review, lalu diletakan di kolom baru bernama 'SCORES_VADER'

kita lihat tabel yang baru, bandingkan dengan table lama

In [21]:
amz_review.head(10)

,review,label,scores_VADER
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0,-0.3535
1,"Good case, Excellent value.",1,0.8402
2,Great for the jawbone.,1,0.6249
3,Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!,0,-0.6145
4,The mic is great.,1,0.6249
5,I have to jiggle the plug to get it to line up right to get decent volume.,0,0.0000
6,"If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.",0,0.5106
7,If you are Razr owner...you must have this!,1,0.0000
8,"Needless to say, I wasted my money.",0,-0.4939
9,What a waste of money and time!.,0,-0.4753


Saya ingin bukan compound score nya tapi cuman apakah sentimen nya positif atau negative. Coding dibawah ini untuk mengassign kalau score dibawah nol berarti negative dan dikasih label 0 dan jika compound score nya diatas nol berarti sentiment positive dan dikasih label 1

In [23]:
# mengassign positive (1) atau negative (0) berdasar nilai scores_vader dan diletakan di kolom baru 'pred_VADER'
amz_review['pred_VADER'] = amz_review['scores_VADER'].apply(lambda x: 1 if x >=0 else 0)
amz_review.head(10)

,review,label,scores_VADER,pred_VADER
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0,-0.3535,0
1,"Good case, Excellent value.",1,0.8402,1
2,Great for the jawbone.,1,0.6249,1
3,Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!,0,-0.6145,0
4,The mic is great.,1,0.6249,1
5,I have to jiggle the plug to get it to line up right to get decent volume.,0,0.0000,1
6,"If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.",0,0.5106,1
7,If you are Razr owner...you must have this!,1,0.0000,1
8,"Needless to say, I wasted my money.",0,-0.4939,0
9,What a waste of money and time!.,0,-0.4753,0


Bisa kita lihat mayoritas cocok antara 'pred_VADER' dengan label manual 'label'

Oleh karena itu kita ingin menghitung berapa persen akurasi dari Vader sentiment analysis. Yaitu seberapa besar cocok dengan manual label

In [24]:
# Compare Actual and Predicted
accuracy_score(amz_review['label'],amz_review['pred_VADER'])

0.768

Accuracy nya ternyata 76,8 %

In [25]:
!pwd


/content/drive/MyDrive/NLP2023
